In [2]:
import numpy as np
import pandas as pd

In [66]:
wheel = pd.read_csv('assets/westland_wheel_csv.csv')
wheel

,cat_id,cat_name,note_id,note_name,desc_id,desc_name
0,1,floral,11,exotic,111,orange_blossom
1,1,floral,11,exotic,112,carnation
2,1,floral,11,exotic,113,honeysuckle
3,1,floral,11,exotic,114,jasmine
4,1,floral,12,garden,121,mint
...,...,...,...,...,...,...
158,7,peaty,73,smoky,734,campfire
159,7,peaty,74,medicinal,741,hospital
160,7,peaty,74,medicinal,742,iodine
161,7,peaty,74,medicinal,743,tar


In [67]:
ratings = pd.read_csv('assets/wh_ratings_csv.csv')


In [68]:
ratings = ratings.fillna(999) # anything beginning with 9 identifies a missing value

In [69]:
ratings['cat_id'] = ratings.desc_id.astype(str).str[:1].astype(int)

ratings

,cust_id,wh_id,wh_rating,cat_id,note_id,desc_id,other_desc
0,1000,101,2.0,7,999.0,743.0,999
1,1000,101,2.0,5,999.0,534.0,999
2,1000,101,2.0,9,999.0,999.0,alcohol
3,1000,101,2.0,2,999.0,211.0,999
4,1000,102,3.0,3,999.0,315.0,999
...,...,...,...,...,...,...,...
330,1022,103,4.0,3,999.0,315.0,999
331,1022,103,4.0,9,35.0,999.0,999
332,1022,103,4.0,9,999.0,999.0,complex
333,1022,103,4.0,9,999.0,999.0,bbq sauce


In [141]:
ratings['note_id'] = ratings.desc_id.astype(str).str[:2].astype(int)
ratings['desc_id']=ratings['desc_id'].astype(int)
ratings['wh_id']=ratings['wh_id'].astype(str)

In [470]:
ratings_wheel = pd.merge(ratings, wheel, on=['desc_id', 'cat_id', 'note_id'] )
ratings_wheel

,cust_id,wh_id,wh_rating,cat_id,note_id,desc_id,other_desc,cat_name,note_name,desc_name
0,1000,101,2.0,7,74,743,999,peaty,medicinal,tar
1,1000,101,2.0,5,53,534,999,sulphur,rubber,pencil_eraser
2,1000,101,2.0,2,21,211,999,fruity,artificial,nail_polish_remover
3,1000,102,3.0,3,31,315,999,gourmand,cream,vanilla_bean
4,1000,103,4.0,3,31,315,999,gourmand,cream,vanilla_bean
...,...,...,...,...,...,...,...,...,...,...
126,1016,101,3.5,4,45,454,999,grainy,nuts,peanut
127,1018,101,999.0,4,45,454,999,grainy,nuts,peanut
128,1016,104,1.0,1,14,141,999,floral,field,barn
129,1017,101,3.5,1,14,141,999,floral,field,barn


In [445]:
smaller_ratings = ratings_wheel[['cust_id', 'wh_id', 'wh_rating','cat_name', 'note_name', 'desc_name']]

In [446]:
new_df=smaller_ratings.groupby(['wh_id', 'cat_name', 'note_name', 'desc_name','wh_rating']).agg(
    votes=pd.NamedAgg(column="desc_name", aggfunc="count"))

In [447]:
new_df.reset_index(inplace=True)

In [489]:
new_df.replace(to_replace=999.0, value=np.NaN, inplace=True)


In [490]:
new_df

,wh_id,cat_name,note_name,desc_name,wh_rating,votes
0,101,floral,field,barn,3.5,1
1,101,floral,field,tobacco_leaf,4.0,1
2,101,floral,woodland,cedar,NaN,1
3,101,floral,woodland,sandalwood,NaN,1
4,101,fruity,artificial,nail_polish_remover,2.0,1
...,...,...,...,...,...,...
98,104,peaty,smoky,peat_reek,3.0,1
99,104,peaty,smoky,peat_reek,4.0,2
100,104,sulphur,rubber,rubber_tires,2.0,1
101,104,woody,lumber,oaky,3.0,1


In [491]:
one = pd.DataFrame(new_df[:28])
two = pd.DataFrame(new_df[28:64]).reset_index(drop=True)
three = pd.DataFrame(new_df[64:89]).reset_index(drop=True)
four = pd.DataFrame(new_df[89:]).reset_index(drop=True)

In [500]:
four.wh_rating.mean()

2.5384615384615383

In [469]:
len(one)+len(two)+len(three)+len(four)

103

In [355]:
import plotly.express as px
import plotly.graph_objects as go

In [496]:
fig1 = px.sunburst(one,
                  path=one[['cat_name', 'note_name', 'desc_name']],
                  values='votes',
                  title="Whiskey #1: Mean Rating = 3.25",
                  width=750, height=750,
                  color_discrete_sequence=px.colors.qualitative.Antique)
fig1.show()

In [497]:
fig2 = px.sunburst(two,
                  path=two[['cat_name', 'note_name', 'desc_name']],
                  values='votes',
                  title="Whiskey #2: Mean Rating = 3.39",
                  width=750, height=750,
                  color_discrete_sequence=px.colors.qualitative.Antique)
fig2.show()

In [499]:
fig3 = px.sunburst(three,
                  path=three[['cat_name', 'note_name', 'desc_name']],
                  values='votes',
                  title="Whiskey #3: Mean Rating = 3.74",
                  width=750, height=750,
                  color_discrete_sequence=px.colors.qualitative.Antique)
fig3.show()

In [395]:
colors = ['#774F38','#E08E79','#ECE5CE','#C5E0DC',
          '#F1D4AF', '#989DBD', '#6786BC', '#E7C4A6', '#F3BDA3', '#C2A2A5']

In [501]:
fig4 = px.sunburst(four,
                  path=four[['cat_name', 'note_name', 'desc_name']],
                  values='votes',
                  title="Whiskey #4: Mean Rating = 2.54",
                  width=750, height=750,
                 color_discrete_sequence=px.colors.qualitative.Antique)
fig4.show()